In [ ]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import folium
import os
import webbrowser
from folium import plugins
import matplotlib.font_manager as fm
pd.plotting.register_matplotlib_converters()
# 가능한 font list 확인
# f = [f.name for f in fm.fontManager.ttflist] # 가능한 폰트 출력
plt.rc('font', family='Malgun Gothic')

seoul = [37.562225, 126.978555] # 지도 생성 시 기준 위치 설정

In [ ]:
# csv 파일 읽어와서 데이터 정리
# 1. 5대 범죄 발생 현황
crime_report = pd.read_csv("./5대 범죄 발생 현황.csv", index_col = 0)
crime_data = crime_report.iloc[4:, 0:2].reset_index(drop = True) # 범죄 합계만 추출
crime_data.columns = ['name', 'values'] # 이름 정리
crime_data.sort_values(by = 'name') # 자치구 정렬

# object 타입의 데이터 float 형식으로 변경
crime_data.set_index('name', inplace = True) # index로 지정된 열은 데이터 타입 변경에서 제외
crime_data = crime_data.astype(float)
crime_data.reset_index(inplace = True)

# 서울 지도 생성
crime_map = folium.Map(location=seoul, tiles="OpenStreetMap", zoom_start=11)

# # 서울 행정구역 데이터
state_latlng = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

folium.Choropleth(
    geo_data = state_latlng,
    name = 'choropleth',
    data = crime_data,
    columns = ['name', 'values'],
    key_on = 'feature.properties.name',
    fill_color = 'OrRd',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    color = 'red'
).add_to(crime_map)

crime_map

In [ ]:
# 2. CCTV 정보 (생활방범용, 다목적, 어린이보호 cctv만 출력)
cctv = pd.read_csv("./CCTV 정보.csv", encoding = 'cp949', index_col = 0)
cctv_info = cctv.loc[(cctv['설치목적구분']=='생활방범')|(cctv['설치목적구분']=='다목적')|(cctv['설치목적구분']=='어린이보호')]

# 위도, 경도, 주소 따로 뽑아서 인덱스 정리
cctv_info_lat = cctv_info.loc[:,'WGS84위도'].reset_index(drop = True) # 기존 인덱스 제거
cctv_info_lng = cctv_info.loc[:,'WGS84경도'].reset_index(drop = True) 

# 순서대로 위도, 경도 뽑기 .iloc[i]
cctv_map = folium.Map(location=seoul, zoom_start = 11)
cctv_map.add_child(folium.LatLngPopup())

# folium.Circle([cctv_info_lat[1], cctv_info_lng[1]], radius = 30, color = '#eb9e34', fill_color = 'red', popup = 'CCTV').add_to(map)
# map
for i in range(cctv_info_lat.count()):
    folium.Circle([cctv_info_lat[i], cctv_info_lng[i]], radius = 30, color = '#eb9e34', fill_color = 'red', popup = 'CCTV').add_to(cctv_map)

cctv_map

In [ ]:
# 2-1. CCTV 정보 (서울시 안심이 CCTV)
cctv_seoul = pd.read_csv("./서울시 안심이 CCTV 연계 현황.csv", encoding = 'cp949')

# 위도, 경도 따로 뽑아서 인덱스 정리
cctv_seoul_lat = cctv_seoul.loc[:,'위도']
cctv_seoul_lng = cctv_seoul.loc[:,'경도']

# 순서대로 위도, 경도 뽑기 .iloc[i]
cctv_seoul_map = folium.Map(location=seoul, zoom_start = 11)
cctv_seoul_map.add_child(folium.LatLngPopup())

for i in range(cctv_seoul_lat.count()):
    folium.Circle([cctv_seoul_lat[i], cctv_seoul_lng[i]], radius = 30, color = '#eb9e34', fill_color = 'red', popup = 'CCTV').add_to(cctv_seoul_map)

cctv_seoul_map

In [ ]:
# 2-2. CCTV 정보 (서울시 안심이 CCTV, 자치구별 표시)
cctv_region = cctv_seoul.value_counts(cctv_seoul.자치구).reset_index().sort_values(by = '자치구')

# 서울 지도 생성
cctv_region_map = folium.Map(location=seoul, tiles="OpenStreetMap", zoom_start=11)

# 서울 행정구역 데이터
state_latlng = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

folium.Choropleth(
    geo_data = state_latlng,
    name = 'choropleth',
    data = cctv_region,
    columns = ['자치구', 'count'],
    key_on = 'feature.properties.name',
    fill_color = 'Blues',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    color = 'blue'
).add_to(cctv_region_map)

cctv_region_map

In [ ]:
# 2-2. CCTV 설치 수 대비 범죄 발생 비율
cctv_region = cctv_region.reset_index(drop = True) # 자치구별 CCTV 설치 대수
crime_data = crime_data.sort_values(by = 'name').reset_index(drop = True) # 자치구별 범죄 발생 횟수

# 자치구만 출력해서 리스트 저장
region = cctv_region['자치구'].to_list()

num = cctv_region['자치구'].count() # 행 개수

# cctv 설치 수 대비 범죄 발생 비율 계산
crime_to_cctv = []

for i in range(num):
    rate = crime_data.loc[i, 'values'] / cctv_region.loc[i, 'count'] * 100
    crime_to_cctv.append(rate)

crime_to_cctv

data = {
    '자치구' : region,
    'rate' : crime_to_cctv
}

crime_to_cctv = pd.DataFrame(data, columns = ['자치구', 'rate'], index = region)

# sns.scatterplot(x = cctv_region['count'], y = crime_data['values'])

# 서울 지도 생성
crime_to_cctv_map = folium.Map(location=seoul, tiles="OpenStreetMap", zoom_start=11)

# 서울 행정구역 데이터
state_latlng = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

folium.Choropleth(
    geo_data = state_latlng,
    name = 'choropleth',
    data = crime_to_cctv,
    columns = ['자치구', 'rate'],
    key_on = 'feature.properties.name',
    fill_color = 'Reds',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    color = 'red'
).add_to(crime_to_cctv_map)

crime_to_cctv_map


In [ ]:
# 3. CCTV 정보 
safety_bell = pd.read_csv("./CCTV 정보.csv", encoding = 'cp949', index_col = 0)

# 위도, 경도만 뽑아서 인덱스 정리
safety_bell_lat = safety_bell.loc[:, 'WGS84위도'].reset_index(drop = True) # 기존 인덱스 제거
safety_bell_lng = safety_bell.loc[:,'WGS84경도'].reset_index(drop = True) # 기존 인덱스 제거

# 순서대로 위도, 경도, 주소 뽑기 .iloc[i]
bell_map = folium.Map(location=seoul, zoom_start = 11)
bell_map.add_child(folium.LatLngPopup())

for i in range(safety_bell_lat.count()):
    folium.Circle([safety_bell_lat[i], safety_bell_lng[i]], radius = 30, color = '#34ebc6', fill_color = 'blue', popup = 'CCTV').add_to(bell_map)

bell_map

In [ ]:
# 4. 안전 시설물 정보
safety_facility = pd.read_csv("./서울시 안심귀갓길 안전시설물.csv", encoding = 'cp949')
safety_facility = safety_facility.drop_duplicates('포인트 wkt ', keep="last")

# POINT(126.968590563668 37.5793826677127) 를 위도와 경도만 남게 분리해서 정리
# lat_lng = safety_facility.loc[3, '포인트 wkt '].strip("POINT("")").split()
num = safety_facility['포인트 wkt '].count()
# safety_facility['포인트 wkt '] = pd.to_numeric(safety_facility['포인트 wkt '], errors='coerce') # 데이터 형식 float 혹은 int로 변경

facility = safety_facility.loc[:, '포인트 wkt '].reset_index(drop = True)

lat_list = []
lng_list = []

for i in range(num):
    lat_lng = facility.iloc[i].strip("POINT("")").split(" ")
    lat_list.append(lat_lng[1]) # 위도와 경도 각각 리스트에 추가
    lng_list.append(lat_lng[0])

# string을 float로 형변환
lat_list_float = list(map(float, lat_list))
lng_list_float = list(map(float, lng_list))

# 순서대로 위도, 경도, 주소 뽑기 .loc[i, '위도'] .loc[i, '경도']
f_map = folium.Map(location=seoul, zoom_start = 11)
f_map.add_child(folium.LatLngPopup())

for i in range(len(lat_list_float)):
    folium.Circle([lat_list_float[i], lng_list_float[i]], radius = 30, color = 'red', fill_color = '#34ebc6', popup = 'facility').add_to(f_map)

f_map

In [ ]:
# geopy를 통해 위도 경도 데이터 가져오기
!pip install geopy

In [ ]:
# 5. 경찰서 위치
police = pd.read_csv("./경찰청_경찰관서 위치(서울청).csv", encoding = 'cp949', index_col = 0)

police_name = (police.loc[:, '관서명'] + police.loc[:, '구분'])
police_location = police.loc[:, '주소']

# 도로명주소를 위도 경도 값으로 변경
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent = 'South Korea')

# 위도, 경도 반환하는 함수. 값이 없을 경우 [0,0] 반환
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y
    
    except:
        return [0,0]
    
latitude = []
longitude = []
for i in police_location:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])


In [ ]:
# 경찰서 위도 경도 좌표 지도에 표시

# 순서대로 위도, 경도, 주소 뽑기 .loc[i, '위도'] .loc[i, '경도']
police_map = folium.Map(location=seoul, zoom_start = 11)
police_map.add_child(folium.LatLngPopup()) # 지도 클릭으로 위도, 경도 표시

for i in range(len(latitude)):
    folium.Circle([latitude[i], longitude[i]], radius = 1000, color = 'orange', fill_color = 'pink', popup = 'police').add_to(police_map)

police_map

In [ ]:
# 5-1. 경찰관서 자치구별 지도 표시
# ex : 서울특별시 중구 을지로 234 -> 띄어쓰기 기준 주소 분리
police['주소'].str.split(" ", expand = True)

# 자치구만 뽑은 column 생성
police['자치구'] = police['주소'].str.split(" ", expand = True)[1]

# 자치구 개수
police_region = police['자치구'].value_counts().reset_index().sort_values(by = '자치구')

# 서울 지도 생성
police_region_map = folium.Map(location=seoul, tiles="OpenStreetMap", zoom_start=11)

# 서울 행정구역 데이터
state_latlng = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

folium.Choropleth(
    geo_data = state_latlng,
    name = 'choropleth',
    data = police_region,
    columns = ['자치구', 'count'],
    key_on = 'feature.properties.name',
    fill_color = 'YlOrBr',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    color = 'blue'
).add_to(police_region_map)

police_region_map

In [ ]:
# 5-2. 경찰관서 대비 범죄 발생 비율
crime_data
police_region = police_region.reset_index(drop = True)

# 자치구만 출력해서 리스트 저장
region = cctv_region['자치구'].to_list()

num = crime_data['name'].count() # 데이터 개수

crime_to_police = []
# 경찰관서 대비 범죄 발생 비율 계산
for i in range(num):
    rate = crime_data.loc[i, 'values'] / police_region.loc[i, 'count']
    crime_to_police.append(rate)

crime_to_police

data = {
    '자치구' : region,
    'rate' : crime_to_police
}

crime_to_police = pd.DataFrame(data, columns = ['자치구', 'rate'], index = region)

# sns.scatterplot(x = cctv_region['count'], y = crime_data['values'])

# 서울 지도 생성
crime_to_police_map = folium.Map(location=seoul, tiles="OpenStreetMap", zoom_start=11)

# 서울 행정구역 데이터
state_latlng = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

folium.Choropleth(
    geo_data = state_latlng,
    name = 'choropleth',
    data = crime_to_police,
    columns = ['자치구', 'rate'],
    key_on = 'feature.properties.name',
    fill_color = 'BuPu',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    color = 'puple'
).add_to(crime_to_police_map)

crime_to_police_map

In [ ]:
# 6. 연도별 방범용 CCTV
cctv_annual = pd.read_csv("./서울시 자치구 연도별 방범용 CCTV 운영 현황.csv", encoding = 'cp949', index_col = '구분')

cctv_data = cctv_annual.iloc[:,1:]
name = cctv_data.index # 자치구 이름 리스트 저장

# 꺾은선 그래프로 나타내기
plt.figure(figsize = (7, 7))
for i in range(cctv_data['2015년'].count()):
    years = ['2015년', '2016년', '2017년', '2018년', '2019년', '2020년', '2021년', '2022년', '2023년']
    num = cctv_annual.iloc[i, 1:].to_list()
    plt.plot(years, num, label = name[i])
    plt.legend(loc=[1.02,0], frameon=True, shadow=True)
    plt.xlabel('years')
    plt.ylabel('CCTV 설치 갯수')
    plt.title('연도, 자치구별 CCTV 설치 갯수')


In [ ]:
# 7. 범죄 발생 현황(~2018년)
crime = pd.read_csv("./범죄+발생현황_20231128105906.csv", index_col = 0)
crime = crime.loc[['발생', '검거'], ['2018','2019','2020','2021','2022']]

crime = crime.astype(float) # 데이터 타입 object -> float 변경

# 검거율 계산 후 리스트 저장
crime_rate = []
for i in range(5):
    rate = crime.iloc[1, i] / crime.iloc[0, i] * 100
    crime_rate.append(rate)

# 데이터프레임에 검거율 행 추가
crime.loc['검거율'] = crime_rate

# 꺾은선 그래프로 나타내기
years = ['2018년','2019년','2020년','2021년','2022년']
plt.plot(years, crime_rate)
plt.xlabel('years')
plt.ylabel('검거율(%)')
plt.ylim(65,80)
plt.title('연도별 범죄 검거율')


In [ ]:
# 꺾은선 그래프로 나타내기
crime_number = crime.iloc[0,:]

years = ['2018년','2019년','2020년','2021년','2022년']
plt.plot(years, crime_number)
plt.xlabel('years')
plt.ylabel('범죄 발생 횟수')
plt.title('연도별 범죄 발생 건')

In [ ]:
# 8. 범죄 발생 장소
crime_place = pd.read_csv("./5대범죄+발생장소별+현황_20231128105943.csv", index_col = 0)
kind = crime_place.iloc[3:, 0].to_list() # 범죄 발생 종류 추출 후 리스트로 형변환
kind_number = crime_place.iloc[3:, -14].to_list() # 범죄 발생 횟수 추출 후 리스트로 형변환
place = crime_place.iloc[1, -13:].to_list() # 범죄 발생 장소만 추출 후 리스트로 형변환
crime_place = crime_place.replace('-', '0') # - 값 0으로 변경
crime_place = crime_place.iloc[2:, 1:].astype(float) # str 값 float로 타입 변경

crime_place_2022 = crime_place.iloc[0,-13:].to_list() # 2022년 기준 범죄 발생 장소별 소계만 추출 및 리스트로 형변환

total = crime_place.iloc[0,-14] # 2022년 발생한 범죄 합계

# 범죄 발생 장소별 총 발생 횟수 대비 비율 계산 후 저장
crime_number = []
for i in range(len(crime_place_2022)):
    number = crime_place_2022[i] / total * 100
    crime_number.append(number)

# 파이차트 생성
wedgeprops = {'width' : 0.7, 'linewidth' : 1}
explode = [0, 0, 0.1, 0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
colors = ['red', 'yellow', 'purple', 'burlywood', 'lightcoral', 'goldenrod', 'pink', 'skyblue', 'blue', 'cyan', 'lightgray', 'lime', 'coral']

plt.pie(crime_number,
    autopct = '%.1f%%',
    explode = explode,
    wedgeprops = wedgeprops,
    shadow = True,
    startangle = 90, # 시작점 90도 지정
    counterclock = False, # 시계방향으로 그래프
    colors = colors)

plt.title('범죄 발생 장소(2022년)', size = 15)
plt.legend(labels = place, loc = [1, 0.1], shadow = True)
plt.show()

In [ ]:
# 8-1. 범죄 종류
number = list(map(float, kind_number)) # str -> float 데이터 타입 변환

# 파이차트 생성
wedgeprops = {'width' : 0.7, 'linewidth' : 1}
explode = [0, 0, 0, 0.1, 0.1]
colors = ['red', 'purple', 'skyblue', 'blue', 'cyan']

plt.pie(number,
    autopct = '%.1f%%',
    explode = explode,
    wedgeprops = wedgeprops,
    shadow = True,
    startangle = 90, # 시작점 90도 지정
    counterclock = False, # 시계방향으로 그래프
    colors = colors)

plt.title('발생한 범죄 종류(2022년)', size = 15)
plt.legend(labels = kind, loc = [1, 0.4], shadow = True)
plt.show()
